# Hands On #2 - Skating the Frozen Lake w/Monte Carlo

## Goal:
* Implement Monte Carlo
* Explore different policies solving OpenAIGym : FrozenLake

## Steps:
1. Program Monte Carlo – 1st Visit or every visit
2. Run the code and understand how it all comes together
3. How is the exploration policy implemented ?
4. How do we finally get the learned policy from the agent  ?
5. Does it correspond to our deterministic policy ?

### Reference : 
* Based on Udacity github https://github.com/udacity/deep-reinforcement-learning/tree/master/monte-carlo
* plus my solution for the DQN https://github.com/xsankar/DQN_Navigation/blob/master/Navigation-v2.ipynb

### 1. Install the required packages

* No esoteric requirements
* You can run them without docker
* pip install -r requirements.txt
* Requirements
 * python 3.6, pytorch, openAI gym, numpy, matplotlib
 * anaconda is easier but not needed
 * Miniconda works fine

### 2.0. Define imports

python 3, numpy, matplotlib, torch, gym

In [1]:
# General imports
import gym

import sys
import numpy as np
import random
from collections import namedtuple, deque, defaultdict

import matplotlib.pyplot as plt
%matplotlib inline

# torch imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

### 2.1. Global Constants and other variables

In [2]:
# Constants Definitions
BUFFER_SIZE = int(1e5)  # replay buffer size
BATCH_SIZE = 64         # minibatch size
GAMMA = 0.99            # discount factor
TAU = 1e-3              # for soft update of target parameters
LR = 5e-4               # learning rate 
UPDATE_EVERY = 4        # how often to update the network
# Number of neurons in the layers of the Q Network
FC1_UNITS = 16
FC2_UNITS = 8
FC3_UNITS = 4
# Store models flag. Store during calibration runs and do not store during hyperparameter search
STORE_MODELS = False

### Work Area

In [3]:
# Work area to quickly test utility functions
import time
from datetime import datetime, timedelta
start_time = time.time()
time.sleep(10)
print('Elapsed : {}'.format(timedelta(seconds=time.time() - start_time)))

Elapsed : 0:00:10.001298


In [4]:
print("State :")
print(" 0  1  2  3")
print(" 4  5  6  7")
print(" 8  9 10 11")
print("12 13 14 15")
print()
print(" S  F  F  F")
print(" F  H  F  H")
print(" F  F  F  H")
print(" H  F  F  G")

State :
 0  1  2  3
 4  5  6  7
 8  9 10 11
12 13 14 15

 S  F  F  F
 F  H  F  H
 F  F  F  H
 H  F  F  G


### Slippery Slope !
#### The default FrozenLake-v0 is very slippery. Let us create an environment that is not slippery.

In [5]:
from gym.envs.registration import register
register(
    id='FrozenLakeNotSlippery-v0',
    entry_point='gym.envs.toy_text:FrozenLakeEnv',
    kwargs={'map_name' : '4x4', 'is_slippery': False},
    max_episode_steps=100,
    reward_threshold=0.78, # optimum = .8196
)

In [6]:
import gym
env = gym.make('FrozenLakeNotSlippery-v0')
env.reset()
env.render()

/miniconda3/lib/python3.7/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)



SFFF
FHFH
FFFH
HFFG


### 3. Examine the State and Action Spaces

The state space is 16, with four actions [ 0 = Left, 1 = Down, 2 = Right, 3 = Up ]

In [7]:
print(env.observation_space)
print(env.action_space)
act_space = [i for i in range(0,env.action_space.n)]
print(act_space)
# env.unwrapped.get_action_meanings() # AttributeError: 'FrozenLakeEnv' object has no attribute 'get_action_meanings'
print('[ 0 = Left, 1 = Down, 2 = Right, 3 = Up ]')

Discrete(16)
Discrete(4)
[0, 1, 2, 3]
[ 0 = Left, 1 = Down, 2 = Right, 3 = Up ]


In [8]:
#print(dir(env))
#print(dir(env.unwrapped))
print('States = {:d}'.format(env.unwrapped.nS))
print('Actions = {:d}'.format(env.unwrapped.nA))
print(env.unwrapped.P[0])

States = 16
Actions = 4
{0: [(1.0, 0, 0.0, False)], 1: [(1.0, 4, 0.0, False)], 2: [(1.0, 1, 0.0, False)], 3: [(1.0, 0, 0.0, False)]}


### Frozen Lake, but slippery no more !

### 4. Test the environment with Random Action

In [9]:
for i_episode in range(3):
    state = env.reset()
    while True:
        action = env.action_space.sample()
        next_state, reward, done, info = env.step(action)
        print('[',state,']',' -> ', action,' = [',next_state,']', 'R = ',reward)
        env.render()
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break
        else:
            state = next_state

[ 0 ]  ->  0  = [ 0 ] R =  0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  1  = [ 4 ] R =  0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  0  = [ 4 ] R =  0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  1  = [ 4 ] R =  0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  1  = [ 4 ] R =  0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  2  = [ 5 ] R =  0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
End game! Reward:  0.0
You lost :(

[ 0 ]  ->  0  = [ 0 ] R =  0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  3  = [ 0 ] R =  0.0
  (Up)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  2  = [ 1 ] R =  0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 1 ]  ->  0  = [ 0 ] R =  0.0
  (Left)
SFFF
FHFH
FFFH
HFFG
[ 0 ]  ->  0  = 

In [10]:
# A Deterministic Optimal Policy
aPolicy = {0:1,4:1,8:2,9:1,13:2,14:2}
# [ 0 = Left, 1 = Down, 2 = Right, 3 = Up ]

<img src='Frozen_Lake_Policy.png'>

In [11]:
for i_episode in range(2): # Should be over in 6 steps, try for 2 episodes
    state = env.reset()
    while True:
        policy_action = aPolicy.get(state,-1)
        if policy_action == -1 :
            action = env.action_space.sample()
        else:
            action = policy_action
        next_state, reward, done, info = env.step(action)
        print('[',state,']',' -> ', action,' = [',next_state,']', reward)
        env.render()
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break
        else:
            state = next_state

[ 0 ]  ->  1  = [ 4 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  1  = [ 8 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 8 ]  ->  2  = [ 9 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 9 ]  ->  1  = [ 13 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 13 ]  ->  2  = [ 14 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 14 ]  ->  2  = [ 15 ] 1.0
  (Right)
SFFF
FHFH
FFFH
HFFG
End game! Reward:  1.0
You won :)

[ 0 ]  ->  1  = [ 4 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  1  = [ 8 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 8 ]  ->  2  = [ 9 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 9 ]  ->  1  = [ 13 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 13 ]  ->  2  = [ 14 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 14 ]  ->  2  = [ 15 ] 1.0
  (Right)
SFFF
FHFH
FFFH
HFFG
End game! Reward:  1.0
You won :)



## Monte Carlo
### Let us implement the Basic MonteCarlo Algorithm
<img src='Montecarlo_Alg.png'>

### Step 1 : Define policies

## $\epsilon$-Greedy

<img src="e_greedy.png" >

In [12]:
def epsilon_greedy(Q,epsilon,nA):
    policy = {}
    for k in Q:
        rand = np.random.random_sample()
        if rand <= epsilon:
            probs = [1.0/nA] * nA
        else: # Exploit ie choose the highest action
            probs = np.zeros(nA)
            probs[np.argmax(Q[k])] = 1
        policy[k] = probs
    return policy

In [13]:
num_states = env.unwrapped.nS
num_actions = env.unwrapped.nA
epsilon_start = 1.0
epsilon_min = 0.1
epsilon_val = 0.3
epsilon_alpha = 0.95

In [14]:
import array
def generate_episode_from_policy(env,policy):
    episode = []
    state = env.reset()
    while True:
        if state in policy:
            probs = policy[state]
        else:
            probs = [1.0/num_actions] * num_actions
        action = np.random.choice(np.arange(num_actions), p=probs)
        next_state, reward, done, info = env.step(action)
        # Reward shaping
        if done:
            if reward == 1: # Goal
                reward = 10 
            else: # Hole
                reward = -10
        episode.append((state, action, reward))
        state = next_state
        if done:
            break
    return episode

In [15]:
def mc_basic(env, num_episodes, alpha=0.9):
    # initialize empty dictionaries of arrays
    returns_sum = defaultdict(lambda: np.zeros(env.action_space.n))
    N = defaultdict(lambda: np.zeros(env.action_space.n))
    Q = defaultdict(lambda: np.zeros(env.action_space.n))
    epsilon = epsilon_start
    # loop over episodes
    for i_episode in range(1, num_episodes+1):
        # monitor progress
        if i_episode % 1000 == 0:
            print("\rEpisode {}/{}.".format(i_episode, num_episodes), end="")
            sys.stdout.flush()
        
        ## decay epsilon as a factor of the episodes
        epsilon = 1.0/(i_episode+0.5)
        if (epsilon < epsilon_min):
            epsilon = epsilon_min 
        #
        # Or we could do an exponential decay
        # epsilon = epsilon * epsilon_alpha
        #
        policy = epsilon_greedy(Q,epsilon,num_actions)
        episode = generate_episode_from_policy(env,policy)
        # Every-visit MC Prediction
        g_t = 0
        for i_visit in episode:
            g_t += i_visit[2]
        for i_visit in episode:
            N[i_visit[0]][i_visit[1]] += 1.0
            returns_sum[i_visit[0]][i_visit[1]] += g_t #i_visit[0][2]
            Q[i_visit[0]][i_visit[1]] = returns_sum[i_visit[0]][i_visit[1]] / N[i_visit[0]][i_visit[1]]
        # Replace the above for loop with the following one liner for constant-alpha MC control
        #for i_visit in episode:
        #   Q[i_visit[0]][i_visit[1]] = Q[i_visit[0]][i_visit[1]] + (alpha * (g_t - Q[i_visit[0]][i_visit[1]]))

    return policy, Q

In [16]:
import time
from datetime import datetime, timedelta
start_time = time.time()
# obtain the estimated optimal policy and action-value function
policy, Q = mc_basic(env, 10000)
print('Elapsed : {}'.format(timedelta(seconds=time.time() - start_time)))
print(datetime.now())

Episode 10000/10000.Elapsed : 0:00:05.491907
2019-02-28 18:48:52.627869


### Points to Ponder
1. There are different ways of decaying $\epsilon$
2. Or we could just keep $\epsilon$=1, making it uniformly random; but this won't work
 * We will see some interesting results
3. GLIE – decay ε as decreasing function of episodes
4. Exponentially decay ε 
 * The $\epsilon$ decay is to incorporate the learning and make the action more deterministic
5. We could use softmax and keep the policy constant ie weighted random.
 * As it learns, it's belief of the grid world changes and that shows up in the preferences
5. After 1,000,000 episodes, it usually wins all the time.
 * I had the $\epsilon$-min = 0.1. Probably it should be reduced

In [17]:
# print(policy)
for k, v in sorted(policy.items()):
    print(k,"=",v)
for k, v in sorted(Q.items()):
    print(k,'-',v)
#[ 0 = Left, 1 = Down, 2 = Right, 3 = Up ]

0 = [0. 1. 0. 0.]
1 = [1. 0. 0. 0.]
2 = [1. 0. 0. 0.]
3 = [1. 0. 0. 0.]
4 = [0. 1. 0. 0.]
6 = [0. 0. 0. 1.]
8 = [0. 0. 1. 0.]
9 = [0. 0. 1. 0.]
10 = [0. 1. 0. 0.]
13 = [0. 0. 1. 0.]
14 = [0. 0. 1. 0.]
0 - [-9.73252987  5.94728498  1.54867257  2.9707113 ]
1 - [  1.57689812 -10.          -7.73584906  -8.47222222]
2 - [-2.68882175 -4.58823529 -3.79310345 -6.35036496]
3 - [ -3.57142857 -10.         -10.         -10.        ]
4 - [ -9.1367173    6.74067868 -10.          -0.6581741 ]
6 - [-10.          -4.54545455 -10.          -1.67192429]
8 - [ -4.66155811 -10.           7.82788258  -2.28971963]
9 - [  0.92783505   6.1516035    8.23452846 -10.        ]
10 - [ -1.75792507   9.91490396 -10.          -2.48322148]
13 - [-10.          -7.60683761   8.90944499  -6.61971831]
14 - [ 8.1025641   7.44270205 10.          9.57559682]


<img src="Frozen_Lake_Policy.png">

## Test our policy

In [18]:
for i_episode in range(2): # Should be over in 6 steps, try for 2 episodes
    state = env.reset()
    while True:
        if state in policy:
            probs = policy[state]
        else:
            probs = [1.0/num_actions] * num_actions
        action = np.random.choice(np.arange(num_actions), p=probs)
        next_state, reward, done, info = env.step(action)
        print('[',state,']',' -> ', action,' = [',next_state,']', reward)
        env.render()
        if done:
            print('End game! Reward: ', reward)
            print('You won :)\n') if reward > 0 else print('You lost :(\n')
            break
        else:
            state = next_state

[ 0 ]  ->  1  = [ 4 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  1  = [ 8 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 8 ]  ->  2  = [ 9 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 9 ]  ->  2  = [ 10 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 10 ]  ->  1  = [ 14 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 14 ]  ->  2  = [ 15 ] 1.0
  (Right)
SFFF
FHFH
FFFH
HFFG
End game! Reward:  1.0
You won :)

[ 0 ]  ->  1  = [ 4 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 4 ]  ->  1  = [ 8 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 8 ]  ->  2  = [ 9 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 9 ]  ->  2  = [ 10 ] 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG
[ 10 ]  ->  1  = [ 14 ] 0.0
  (Down)
SFFF
FHFH
FFFH
HFFG
[ 14 ]  ->  2  = [ 15 ] 1.0
  (Right)
SFFF
FHFH
FFFH
HFFG
End game! Reward:  1.0
You won :)



In [20]:
env.close()

### _That's All, Folks !_